In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Load the training data
train_data = pd.read_csv('../input/icr-identify-age-related-conditions/train.csv')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Fill missing values with the median of the respective columns
train_data_filled = train_data.fillna(train_data.median())

# Perform binary encoding on the 'EJ' column
train_data_filled['EJ'] = train_data_filled['EJ'].map({'A': 0, 'B': 1})

# Separate features and target
X = train_data_filled.drop(['Id', 'Class'], axis=1)
y = train_data_filled['Class']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


/tmp/ipykernel_20/2083705546.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_data_filled = train_data.fillna(train_data.median())


In [3]:
# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = model.predict_proba(X_val)[:, 1]

# Calculate the ROC AUC score
roc_auc_score(y_val, val_predictions)

0.9674989238054239

In [4]:
# Load the test data
test_data = pd.read_csv('../input/icr-identify-age-related-conditions/test.csv')

# Fill missing values in the test data with the median of the respective columns
test_data_filled = test_data.fillna(train_data.median())

# Perform binary encoding on the 'EJ' column
test_data_filled['EJ'] = test_data_filled['EJ'].map({'A': 0, 'B': 1})

# Separate the ID column
test_ids = test_data_filled['Id']
test_features = test_data_filled.drop('Id', axis=1)

/tmp/ipykernel_20/232044296.py:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test_data_filled = test_data.fillna(train_data.median())


In [5]:
# Make predictions on the test data
test_predictions = model.predict_proba(test_features)[:, 1]
test_ids = test_ids.astype(str)

# Create a DataFrame with the IDs and predicted probabilities
submission_df = pd.DataFrame({'Id': test_ids, 'Class': test_predictions})

# Write the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)